# Retrieving data via bitcoin-cli

In [ ]:
# install necessary python libraries 
%pip install python-bitcoinrpc
%pip install paramiko

In [ ]:
# read credentials from json file 
import pandas as pd

credentials = pd.read_json('credentials.json')

In [ ]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

## Step 1: Monitor newly incoming files 

In [ ]:
stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblockchaininfo") # returns the header hash of the most recent block on the best block chain.

In [ ]:
# parse json output
import json
blockchaininfo = json.loads(stdout.read().decode("utf-8"))

bestblockhash = blockchaininfo['bestblockhash']
headers = blockchaininfo['headers']
print(bestblockhash)
print(headers)

## Step 2: Get information about the new block

In [ ]:
# get blockinfo 
command = "bitcoin-cli getblock " + bestblockhash + " 2"
stdin, stdout, stderr = ssh.exec_command(command)

block = json.loads(stdout.read().decode("utf-8"))

from datetime import datetime, timezone
ts_epoch = block['time']
block_timestamp = datetime.fromtimestamp(ts_epoch)
block_date = block_timestamp.strftime('%Y-%m-%d')
block_hash = block['hash']
block_height = block['height']
previousblockhash = block['previousblockhash']
# nextblockhash = block['nextblockhash']

# build json object
data = {}
data['block_hash'] = block_hash
data['block_height'] = block_height
data['block_timestamp'] = str(block_timestamp)
data['block_date'] = str(block_date)
data['previousblockhash'] = previousblockhash # relevant for PRECEDES relationship
# data['nextblockhash'] = nextblockhash
json_data = json.dumps(data, indent=4, sort_keys=False)

print(json_data)


## Step 3: Iterate through the list of txids and retrieve information on tx

In [ ]:
# parse transactions which are contained in one block 

import time
startTime = time.time() # measure execution time

for tx in block['tx']: 
    command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    rawtx = json.loads(stdout.read().decode("utf-8"))
    
    address_list = []  

    for o in rawtx['vout']: 
        # check whether there are output adresses
        if 'addresses' in o['scriptPubKey']:
            for a in o['scriptPubKey']['addresses']: 
                # a Python object (dict):
                addrObj = {
                   "addr": a,
                    "outNr": o['n']
                }
                jAddr = json.dumps(addrObj)
                # convert json string to Json Dict
                jsonDict = json.loads(jAddr)
                address_list.append(jsonDict)
        
    # print json object = one transaction within a block
    txdata = {}
    txdata['txid'] = tx['txid']
    txdata['block_hash'] = block_hash
    txdata['block_date'] = str(block_date)
    txdata['address_list'] = address_list
    txdata['inDegree'] = len(rawtx['vin']) 
    txdata['outDegree'] = len(rawtx['vout'])
    tx_json_data = json.dumps(txdata, indent=4, sort_keys=False)

    print(tx_json_data)
    
    
        
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
stdin, stdout, stderr = ssh.exec_command(command)
rawtx = json.loads(stdout.read().decode("utf-8"))
print(rawtx)

In [ ]:
test_list = []
test_list.append('1')
print(test_list)

In [ ]:
test_list = []
# a Python object (dict):
x = {
  "name": "John",
  "age": 30,
  "city": "New York"
}

# convert into JSON string:
y = json.dumps(x)
# convert json string to Json Dict
jsonDict = json.loads(y)
test_list.append(jsonDict)

print(test_list)